# Better CNN model

Here I explore very similar architectures to the baseline (CNN - Flatten - Dense layers)

Hyperparameters experimented with:

- number of CNN and dense layers
- Dropout
- Activation function
- Other max pooling strategies
- learning rate scheduling


After training, model is serialized and uploaded to W&B project.

In [ ]:
import wandb
import tensorflow as tf
import numpy as np
import pathlib
import shutil
import math
from typing import List

def load_data(run, artifact_name = f"phcd_paper_splits_tfds") -> List[tf.data.Dataset]:
    """
    Downloads datasets from a wandb artifact and loads them into a list of tf.data.Datasets.
    """

    artifact = run.use_artifact(f"master-thesis/{artifact_name}:latest")
    artifact_dir = pathlib.Path(
        f"./artifacts/{artifact.name.replace(':', '-')}"
    ).resolve()
    if not artifact_dir.exists():
        artifact_dir = artifact.download()
        artifact_dir = pathlib.Path(artifact_dir).resolve()

    # if tf.__version__ minor is less than 10, use
    # tf.data.experimental.load instead of tf.data.Dataset.load

    if int(tf.__version__.split(".")[1]) < 10:
        load_function = tf.data.experimental.load
    else:
        load_function = tf.data.Dataset.load
    
    output_list = []
    for split in ["train", "test", "val"]:
        ds = load_function(str(artifact_dir / split), compression="GZIP")
        output_list.append(ds)
    
    return output_list

def get_number_of_classes(ds: tf.data.Dataset) -> int:
    """
    Returns the number of classes in a dataset.
    """
    labels_iterator= ds.map(lambda x, y: y).as_numpy_iterator()
    labels = np.concatenate(list(labels_iterator))
    return len(np.unique(labels))

def preprocess_dataset(ds: tf.data.Dataset, batch_size: int, cache: bool = True) -> tf.data.Dataset:
    ds = ds.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))  # normalize
    ds = ds.unbatch().batch(batch_size)
    if cache:
        ds = ds.cache().prefetch(buffer_size=tf.data.AUTOTUNE)
    return ds

def calculate_model_size_on_disk(path: str) -> int:
    return pathlib.Path(path).stat().st_size    

def calculate_model_num_parameters(model: tf.keras.Model) -> int:
    return model.count_params()

def calculate_model_flops(summary) -> float:
    # from run.summary get GFLOPs or GFLOPS whichever is available
    if "GFLOPs" in summary:
        return summary.get("GFLOPs")
    elif "GFLOPS" in summary:
        return summary.get("GFLOPS")


class Terminate_slow_convergence(tf.keras.callbacks.Callback):
    ACCURACY_THRESHOLD =  0.9
    EPOCH = 6
    MAX_VAL_ACCURACY_THRESHOLD = 0.995

    def on_epoch_end(self, epoch, logs={}):
        
        # if model is not converging fast, stop training
        if(logs.get('accuracy') < self.ACCURACY_THRESHOLD and epoch > self.EPOCH):
            print(f"Model accuracy is {logs.get('accuracy')} and is below {self.ACCURACY_THRESHOLD} at epoch {epoch}. Terminating training.")
            self.model.stop_training = True
            
        # if model converged, stop training
        #if(logs.get('val_accuracy') >= self.MAX_VAL_ACCURACY_THRESHOLD and epoch > 2):
        #    self.model.stop_training = True

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
print("Available devices: ", tf.config.list_physical_devices())

In [ ]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2

def get_mobilenet_model(input_shape=(32,32,1), num_classes=89, mobilenet_version = 2 ) -> tf.keras.Sequential:
    base_model = None

    if mobilenet_version == 2:
        base_model = MobileNetV2(include_top=False, input_shape=input_shape, weights=None, pooling = 'max')
    else:
        base_model = MobileNet(include_top=False, input_shape=input_shape, weights=None, pooling = 'max')

    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.Dense(num_classes),
        tf.keras.layers.Activation('softmax', dtype='float32', name='predictions') # this is needed for mixed precision training
    ])
    return model

In [ ]:
def train(model_name, config_defaults, artifact_name, num_classes=89):
    print("Initializing wandb")
    with wandb.init(
        project="master-thesis",
        job_type="training",
        name=model_name,
        config=config_defaults,
    ) as run:
        ds_train, ds_test, ds_val = load_data(run, artifact_name)

        num_classes = get_number_of_classes(ds_val)

        ds_train = preprocess_dataset(ds_train, batch_size=wandb.config.batch_size)
        ds_val = preprocess_dataset(ds_val, batch_size=wandb.config.batch_size)
        ds_test = preprocess_dataset(ds_test, batch_size=wandb.config.batch_size, cache=False)
        
        print(f"Model name: {model_name}")
        model = get_mobilenet_model(num_classes=num_classes, mobilenet_version=2)

        model.compile(
            optimizer='adam',
            loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
            metrics=["accuracy"],
        )

        # save the best model
        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=f"./artifacts/{model_name}.h5",
            save_weights_only=False,
            monitor="val_accuracy",
            mode="max",
            save_best_only=True,
        )

        wandb_callback = wandb.keras.WandbCallback(
            save_model=False,
            compute_flops=True,
        )

        # learning rate scheduler
        initial_learning_rate = wandb.config.epochs
        epochs = wandb.config.epochs
        decay = initial_learning_rate / epochs

        def lr_time_based_decay(epoch, lr):
            return lr * 1 / (1 + decay * epoch)

        def lr_step_decay(epoch, lr):
            drop_rate = 0.5
            epochs_drop = 10.0
            return initial_learning_rate * math.pow(drop_rate, math.floor(epoch/epochs_drop))

        lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_time_based_decay)

        slow_convergence_callback = Terminate_slow_convergence()
        history = model.fit(
            ds_train,
            epochs=wandb.config.epochs,
            validation_data=ds_val,
            callbacks=[wandb_callback, checkpoint_callback, lr_scheduler],
        )
        print("Training finished")

        # calculate model size on disk, flops and number of parameters
        flops = calculate_model_flops(wandb.run.summary)
     
        disk_size = calculate_model_size_on_disk(f"./artifacts/{model_name}.h5")
        num_parameters = calculate_model_num_parameters(model)

        # evaluate model on ds_test and log to wandb
        test_loss, test_acc = model.evaluate(ds_test)

        wandb.log({
            "test loss": test_loss, 
            "test accuracy": test_acc, 
            "number of parameters": num_parameters,
            "disk size": disk_size, 
            "model flops": flops
            })
        # save artifact to wandb
        artifact = wandb.Artifact(name=model_name, type="model")

        # save best model to artifact
        artifact.add_file(f"./artifacts/{model_name}.h5")
        run.log_artifact(artifact)
        run.finish()
        print("Evaluation finished")

In [ ]:
defaults = dict(
    batch_size=128,
    epochs=100,
    learning_rate=0.01
)

train("mobilenet", defaults, artifact_name="uppercase_splits_tfds")